<a href="https://colab.research.google.com/github/kapilb70/CHAT-GPT--2-small-/blob/main/chatbot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install torch transformers flask-ngrok


In [18]:
!wget https://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip cornell_movie_dialogs_corpus.zip


--2024-09-13 08:53:14--  https://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.53
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: ‘cornell_movie_dialogs_corpus.zip.1’

cornell_movie_dialo 100%[===================>]   9.46M  24.8MB/s    in 0.4s    

2024-09-13 08:53:15 (24.8 MB/s) - ‘cornell_movie_dialogs_corpus.zip.1’ saved [9916637/9916637]

Archive:  cornell_movie_dialogs_corpus.zip
replace cornell movie-dialogs corpus/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cornell movie-dialogs corpus/.DS_Store  
replace __MACOSX/cornell movie-dialogs corpus/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
replace cornell movie-dialogs corpus/chameleons.pdf? [y]es, [n]o, [A]ll, [N]one, 

In [19]:
import os

def load_conversations():
    conversations = []
    with open("cornell movie-dialogs corpus/movie_conversations.txt", "r", encoding="iso-8859-1") as file:
        for line in file:
            parts = line.strip().split(" +++$+++ ")
            conv = parts[-1].strip("[]").replace("'", "").replace(" ", "").split(",")
            conversations.append(conv)
    return conversations

conversations = load_conversations()


In [20]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW

class ConversationDataset(Dataset):
    def __init__(self, tokenizer, conversations, max_length=512):
        self.input_ids = []
        for conv in conversations:
            # Tokenize input and output with special tokens
            encoded = tokenizer.encode(f"<|startoftext|>{conv[0]}<|sep|>{conv[1]}<|endoftext|>",
                                       max_length=max_length, truncation=True)
            self.input_ids.append(torch.tensor(encoded))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx]

# Initialize the model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tokenizer adjustments
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'sep_token': '<|sep|>', 'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>'})
model.resize_token_embeddings(len(tokenizer))

# Prepare dataset
dataset = ConversationDataset(tokenizer, conversations)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


In [21]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Pad sequences to the same length
    batch = pad_sequence(batch, batch_first=True, padding_value=tokenizer.pad_token_id)
    return batch


In [22]:
# Create DataLoader with custom collate_fn
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):  # Adjust epochs as needed
    for batch in dataloader:
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Epoch: {epoch}, Loss: {loss.item()}")


Epoch: 0, Loss: 61.208778381347656
Epoch: 0, Loss: 29.54876136779785
Epoch: 0, Loss: 18.80428695678711
Epoch: 0, Loss: 30.01150894165039
Epoch: 0, Loss: 12.876657485961914
Epoch: 0, Loss: 12.670186996459961
Epoch: 0, Loss: 8.749302864074707
Epoch: 0, Loss: 6.602964878082275
Epoch: 0, Loss: 6.3481340408325195
Epoch: 0, Loss: 6.027518272399902
Epoch: 0, Loss: 5.847731113433838
Epoch: 0, Loss: 5.545392990112305
Epoch: 0, Loss: 6.197002410888672
Epoch: 0, Loss: 6.015810489654541
Epoch: 0, Loss: 6.299705505371094
Epoch: 0, Loss: 5.840775489807129
Epoch: 0, Loss: 4.6854166984558105
Epoch: 0, Loss: 4.930420875549316
Epoch: 0, Loss: 4.834855079650879
Epoch: 0, Loss: 4.689339637756348
Epoch: 0, Loss: 3.7997429370880127
Epoch: 0, Loss: 3.8774383068084717
Epoch: 0, Loss: 4.178391456604004
Epoch: 0, Loss: 4.094037055969238
Epoch: 0, Loss: 3.912158250808716
Epoch: 0, Loss: 3.5883116722106934
Epoch: 0, Loss: 3.6871979236602783
Epoch: 0, Loss: 3.6121420860290527
Epoch: 0, Loss: 3.6694259643554688
Epo

In [ ]:
def generate_response(model, tokenizer, prompt, max_length=100, temperature=0.7, top_k=50):
    model.eval()  # Set the model to evaluation mode

    # Encode input with special tokens and create attention mask
    input_ids = tokenizer.encode(f"<|startoftext|>{prompt}<|sep|>", return_tensors='pt').to(device)
    attention_mask = (input_ids != tokenizer.pad_token_id).long()

    # Generate response
    with torch.no_grad():
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            temperature=temperature,
            top_k=top_k,
            pad_token_id=tokenizer.pad_token_id  # Ensure this is set correctly
        )

    # Decode and handle output more robustly
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Ensure there's a separator token before trying to split
    if "<|sep|>" in response:
        return response.split("<|sep|>")[1].strip()
    else:
        return response.strip()  # Return the full response if no separator is found


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Adjust the tokenizer and model for special tokens
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'sep_token': '<|sep|>', 'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>'})
model.resize_token_embeddings(len(tokenizer))

# Ensure the model is on the correct device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [ ]:
while True:
    # Get user input
    user_input = input("You: ")

    # Check for exit conditions
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Chatbot: Goodbye!")
        break

    # Generate chatbot response
    response = generate_response(model, tokenizer, user_input)
    print(f"Chatbot: {response}")
